<a href="https://colab.research.google.com/github/ebisenttt/genetic_algorithm_practice/blob/main/%E6%99%82%E9%96%93%E5%89%B2%E4%BD%9C%E6%88%90_%E9%96%8B%E7%99%BA%E7%92%B0%E5%A2%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [3]:
#変数定義
classes_per_day = 7

#モジュール
import random
import copy
import pprint

#spreadsheetの認証
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

In [4]:
def get_data():
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  #spreadsheetの選択
  ss_id = "1hUENxvhlvel17hPnegIK7jwBaPkt6YENwc-3hcQvpbM"
  workbook = gc.open_by_key(ss_id)

  #希望調査の配列の取得
  sheet_name = "data"
  worksheet = workbook.worksheet(sheet_name)
  data = worksheet.get_all_records()

  return data

In [5]:
def format_data(data):
  id_list = []
  name_list = []
  sum_list = []
  questionnaire_list = []
  schedule_list = []
  for obj in data:
    q = []
    for key, value in obj.items():
      if key == 'id':
        id_list.append(value)
      elif key == 'name':
        name_list.append(value)
      elif key == 'sum':
        sum_list.append(value)
      elif value=='':
        q.append(0)
      else:
        q.append(value)
    questionnaire_list.append(q)
    s = [0 if num != -2 else num for num in q]
    schedule_list.append(s)
  
  return id_list, name_list, sum_list, questionnaire_list, schedule_list

In [13]:
def make_schedule(sum_list, sch_list):
  sch_list_copy = copy.deepcopy(sch_list)
  for sum, sch in zip(sum_list, sch_list_copy):
    while sum != sch.count(1):
      rand_num = random.randint(0, len(sch) - 1)
      if sum > sch.count(1) and sch[rand_num] == 0:
        sch[rand_num] = 1
      elif sum < sch.count(1) and sch[rand_num] == 1:
        sch[rand_num] = 0
      else:
        pass
  return sch_list_copy

In [7]:
def evaluate_schedule(q_list, sch_list):
  score = 0
  for qst, sch in zip(q_list, sch_list):
    for q, s in zip(qst, sch):
      if s == 1:
        if q == -1:
          score += -5
        elif q == 1:
          score += 1
        elif q == 2:
          score += 3
        else:
          pass
      else:
        pass
    for i in range(len(sch) - 1):
      if sch[i-1] == 1 and sch[i] == 1 and sch[i+1] == 1:
        score += -5
  return score

In [66]:
def make_first_generation(sum_list, q_list, sch_list):
  parents = []
  for i in range(100):
    new_sch_list = make_schedule(sum_list, sch_list)
    score = evaluate_schedule(q_list, new_sch_list)
    parent = {
        "score": score,
        "schedule_list": new_sch_list
    }
    parents.append(parent)
  parents.sort(key = lambda parent: parent["score"], reverse=True)

  return parents[0:20]

In [41]:
def crossover(parent0, parent1):
  child0 = []
  child1 = []
  for num0, num1 in zip(parent0, parent1):
    if num0 == num1:
      child0.append(num0)
      child1.append(num1)
    else:
      rand_num = random.randint(0,1)
      child0.append(rand_num)
      child1.append(1^rand_num)
  return child0, child1

In [22]:
def mutation(sch_list):
  rand_mutate = random.randint(0,20)
  if rand_mutate == 0:
    cnt = 0
    num = len(sch_list)*0.1
    while num > cnt:
      rand_num = random.randint(0, len(sch_list)-1)
      s = sch_list[rand_num]
      rand_select = random.randint(0, len(sch_list)-1)
      if s[rand_select] == 1:
        s[rand_select] = 0
        cnt += 1
      elif s[rand_select] == 0:
        s[rand_select] = 1
        cnt += 1
      else:
        pass
  else:
    pass
  return sch_list

In [90]:
def make_next_generation(generation, sum_list, q_list):
  l = len(generation)
  old_strongest = generation[0]
  new_generation = []
  for i in range(l-1):
    for j in range(i+1, l):
      child0_sl = []
      child1_sl = []
      parent0_sl = copy.deepcopy(generation[i]['schedule_list'])
      parent1_sl = copy.deepcopy(generation[j]['schedule_list'])
      for s0, s1 in zip(parent0_sl, parent1_sl):
        c0, c1 = crossover(s0, s1)    
        child0_sl.append(c0)
        child1_sl.append(c1)
    children_sl = [child0_sl, child1_sl]
    for child_sl in children_sl:
      child_sl = mutation(child_sl)
      child_sl = make_schedule(sum_list, child_sl)
      score = evaluate_schedule(q_list, child_sl)
      child = {
          "score": score,
          "schedule_list": child_sl
      }
      new_generation.append(child)
  new_generation.sort(key = lambda parent: parent['score'], reverse=True)
  new_generation_top = new_generation[0:20]
  new_strongest = new_generation_top[0]
  if new_strongest['score'] < old_strongest['score']:
    new_generation_top.append(old_strongest)
    new_generation_top.sort(key = lambda parent: parent['score'], reverse=True)
  return new_generation_top

In [94]:
def pp(item):
  pprint.pprint(item, width = 200, compact = True)

original_data = get_data()
id_list, name_list, sum_list, questionnaire_list, schedule_list = format_data(original_data)
first_parents = make_first_generation(sum_list, questionnaire_list, schedule_list)
next_parents = make_next_generation(first_parents, sum_list, questionnaire_list)
cnt = 0 
prev_strongest = first_parents[0]
strongest = next_parents[0]
while cnt < 50:
  prev_score = prev_strongest['score']
  new_score = strongest['score']
  if prev_score == new_score:
    cnt += 1
  else:
    cnt = 0
  prev_strongest = copy.deepcopy(next_parents[0])
  next_parents = make_next_generation(next_parents, sum_list, questionnaire_list)
  strongest = copy.deepcopy(next_parents[0])
pp(strongest)

{'schedule_list': [[1, 1, 0, 1, 0, -2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, -2, 0, 0, 1, -2, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0],
                   [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, -2, 0, 1, 0, 0, 0, 1, -2, 0, 1, 0, 1, 0, 1],
                   [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, -2, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, -2, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
                   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, -2, 0, 1, 0, 0, 0, 0, 1, 1, 0],
                   [0, 0, 0, 0, 0, -2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, -2, -2],
                   [0, 0, 0, 0, -2, 1, 0, -2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -2, 0, 1, -2, 0, -2, 0, 0, -2, 0, 0, -2, 0, 0, 0]],
 'score': 85}
